## Description

Given $x'(t) = Ax(t)$. Let

$$
 x_c^* := \max_{0 \leq t \leq T} x_c(t) 
$$
where $x_c(t)$ is the temperature at the center of the domain. 

We are interested in the safety problem
$$
u_f-\delta \leq  x_c^* \leq u_f+\delta
$$
where $u_f$ is the final temperature, $\delta = 1e-4$.

## Approach

First we apply an approximation model, getting the recurrence

$$
X_{k+1} = \Phi X_k, \qquad k \geq 0
$$
where $X_0 := \Omega_0$ is the discretized initial condition, and $\Phi := e^{A\delta}$ is the matrix exponential.

To compute the min and max values of $x_c(t)$ over the interval $[0, \delta]$, we can compute the support function we do as follows. Let $e_c$ the canonical vector along the center direction. Then, we compute 

$$
(-\rho(-e_c, X_k), \rho(e_c, X_k)),\qquad k \geq 0.
$$

This is achieved using the LGG09 algorithm with custom directions $e_c$ and $-e_c$.

In [4]:
using MAT, SparseArrays, ReachabilityAnalysis
using ExponentialUtilities
using LazySets.Arrays: SingleEntryVector

In [152]:
# recursive version, default expv
function reach_homog_dir_LGG09!(out, Ω₀, Aᵀ, ℓ, NSTEPS, recursive::Val{:true})
    rᵢ = copy(ℓ) # this is a Vector (!)
    rᵢ₊₁ = similar(rᵢ)

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, Ω₀)

        # update cache for the next iteration
        rᵢ₊₁ = expv(1.0, Aᵀ, rᵢ) # computes exp(Aᵀ * 1.0) * rᵢ
        copy!(rᵢ, rᵢ₊₁)
    end
    return out
end

# ρ(ℓ, Ω₀)
# ρ(exp(Aᵀ) * ℓ, Ω₀)
# ρ(exp(2Aᵀ) * ℓ, Ω₀)
function reach_homog_dir_LGG09!(out, Ω₀, Aᵀ, ℓ, NSTEPS, recursive::Val{:false})
    rᵢ = deepcopy(ℓ) # if ℓ is a sev => this is a sev

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, Ω₀)

        # update cache for the next iteration
        rᵢ = expv(i*1.0, Aᵀ, ℓ)
    end
    return out
end

# non-recursive version using precomputed Krylov subspace
function reach_homog_dir_LGG09c!(out, Ω₀, Aᵀ, ℓ, NSTEPS, recursive::Val{:false})
    rᵢ = deepcopy(ℓ) # if ℓ is a sev => this is a sev
    Ks = arnoldi(Aᵀ, ℓ, tol=1e-18)

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, Ω₀)

        # update cache for the next iteration
        expv!(rᵢ, i*1.0, Ks)
    end
    return out
end

function reach_homog_dir_LGG09h!(out, Ω₀, Aᵀ, ℓ, NSTEPS, recursive::Val{:false};
                                 m=min(30, size(Aᵀ, 1)), tol=1e-7)
    
    TA, Tb = eltype(Aᵀ), eltype(ℓ)
    T = promote_type(TA, Tb)
    Ks = KrylovSubspace{T, real(T)}(length(ℓ), m)
    arnoldi!(Ks, Aᵀ, ℓ; m=m, ishermitian=true, tol=tol)

    rᵢ = deepcopy(ℓ) # if ℓ is a sev => this is a sev

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, Ω₀) # MEJORAR (?)

        # update cache for the next iteration
        expv!(rᵢ, i*1.0, Ks)
    end
    return out
end

reach_homog_dir_LGG09h! (generic function with 1 method)

## HEAT01 (5x5x5 grid)

In [6]:
out = matread("A5.mat");
A = out["A5"]
n = size(A, 1)
xind = sparsevec(out["X0"][:])
x0ind = xind.nzind;
xc = 62 + 1;
NSTEPS = 2_000
δ = 0.02 # step size

c = sparsevec(x0ind, fill(1.0, length(x0ind)), n)
r = sparsevec(x0ind, fill(0.1, length(x0ind)), n)
Ω₀ = Hyperrectangle(c, r);

Aᵀ = transpose(A .* δ); # lazy
ℓ = SingleEntryVector(xc, n, 1.0);

In [8]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09!(out, Ω₀, Aᵀ, ℓ, NSTEPS, Val(:true));
@show maximum(out)

  0.510531 seconds (278.00 k allocations: 135.439 MiB, 8.01% gc time)
maximum(out) = 0.10369885388747664


0.10369885388747664

In [10]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09!(out, Ω₀, Aᵀ, ℓ, NSTEPS, Val(:false));
@show maximum(out)

  0.490708 seconds (278.00 k allocations: 135.437 MiB, 2.05% gc time)
maximum(out) = 0.10369885388747549


0.10369885388747549

In [12]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09c!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:false));
@show maximum(out)

  0.379944 seconds (72.13 k allocations: 49.734 MiB, 0.74% gc time)
maximum(out) = 0.10369885388747549


0.10369885388747549

In [13]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09h!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:false));
@show maximum(out)

  0.443132 seconds (125.79 k allocations: 52.605 MiB, 0.68% gc time)
maximum(out) = 0.10369885388747549


0.10369885388747549

## HEAT02 (10x10x10 grid)

In [111]:
GC.gc()

In [112]:
out = matread("A10.mat");
A = out["A10"]
n = size(A, 1)
xind = sparsevec(out["X0"][:])
x0ind = xind.nzind;
xc = 555 + 1;
NSTEPS = 2_000
δ = 0.02 # step size

c = sparsevec(x0ind, fill(1.0, length(x0ind)), n)
r = sparsevec(x0ind, fill(0.1, length(x0ind)), n)
Ω₀ = Hyperrectangle(c, r);

Aᵀ = transpose(A .* δ); # lazy
ℓ = SingleEntryVector(xc, n, 1.0);

In [113]:
NSTEPS = 1000
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:true));
@show maximum(out)

  9.355580 seconds (142.03 k allocations: 289.204 MiB, 0.10% gc time)
maximum(out) = 0.029334007070958994


0.029334007070958994

In [114]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09h!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:false));
@show maximum(out)

  3.379505 seconds (36.13 k allocations: 25.125 MiB, 0.09% gc time)
maximum(out) = 0.02905704451649251


0.02905704451649251

In [115]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09h!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:false));
@show maximum(out)

  3.198618 seconds (36.13 k allocations: 25.125 MiB)
maximum(out) = 0.02905704451649251


0.02905704451649251

In [110]:
GC.gc()

## HEAT03 (20x20x20 grid)

In [139]:
out = matread("A20.mat");
A = out["A20"]
n = size(A, 1)
xind = sparsevec(out["X0"][:])
x0ind = xind.nzind;
xc = 4210 + 1;
NSTEPS = 2_000
δ = 0.02 # step size

c = sparsevec(x0ind, fill(1.0, length(x0ind)), n)
r = sparsevec(x0ind, fill(0.1, length(x0ind)), n)
Ω₀ = Hyperrectangle(c, r);

Aᵀ = transpose(A .* δ); # lazy
ℓ = SingleEntryVector(xc, n, 1.0);
NSTEPS = 1200

1200

In [167]:
length(x0ind)

135

In [168]:
n

8000

In [166]:
q = rand(n)
@time ρ(q, Ω₀);

  0.000215 seconds (1 allocation: 16 bytes)


In [169]:
@which ρ(q, Ω₀)

ρ(d::AbstractArray{N,1}, H::AbstractHyperrectangle{N}) where N<:Real in LazySets at /home/mforets/.julia/dev/LazySets/src/Interfaces/AbstractHyperrectangle.jl:305

In [ ]:
function foo(d::AbstractVector{N}, H::Hyperrectangle{N}) where {N}
    c = center(H)
    res = zero(N)
    @inbounds for (i, di) in enumerate(d)
        if di < zero(N)
            res += di * (c[i] - radius_hyperrectangle(H, i))
        elseif di > zero(N)
            res += di * (c[i] + radius_hyperrectangle(H, i))
        end
    end
    return res
end

In [129]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:true));
@show maximum(out)

 42.501745 seconds (14.44 k allocations: 205.780 MiB, 0.02% gc time)
maximum(out) = 0.0012942265383708085


0.0012942265383708085

In [153]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09h!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:false); m=94, tol=1e-10);
@show maximum(out)

 96.072502 seconds (88.30 k allocations: 206.125 MiB, 0.04% gc time)
maximum(out) = 0.017165293660651358


0.017165293660651358

## HEAT04 (50 x 50 x 50)

In [175]:
out = matread("A50.mat");
A = out["A50"]
n = size(A, 1)
xind = sparsevec(out["X0"][:])
x0ind = xind.nzind;
xc = 63775 + 1;
δ = 0.02 # step size

c = sparsevec(x0ind, fill(1.0, length(x0ind)), n)
r = sparsevec(x0ind, fill(0.1, length(x0ind)), n)
Ω₀ = Hyperrectangle(c, r);

Aᵀ = transpose(A .* δ); # lazy
ℓ = SingleEntryVector(xc, n, 1.0);
NSTEPS = 1_200

1200

In [176]:
out = Vector{Float64}(undef, NSTEPS)
@time reach_homog_dir_LGG09h!(out, Ω₀, Aᵀ, sparse(ℓ), NSTEPS, Val(:false); m=211, tol=1e-10);
@show maximum(out)

3133.204703 seconds (46.29 k allocations: 1.073 GiB, 0.01% gc time)
maximum(out) = 0.011611334772430324


0.011611334772430324

In [177]:
3133 / 60.

52.21666666666667